In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# ImageDataGenerator: 이미지 증강 및 전처리를 위한 도구
# rescale=1./255: 모든 픽셀 값을 255로 나눠서 0과 1 사이의 값으로 스케일링, 모델 훈련 시 더 나은 성능을 위해 필수적

# shear_range=0.2: 이미지에 랜덤으로 시어 변환을 적용하여 각도 변화를 준다. 이미지의 기울기를 변화시킨다.
#                  이미지를 평행 이동하는 것으로, 이미지의 기울기를 변형 = 다양한 시각적 왜곡에 대해 더 견고
#                  훈련 데이터의 다양성을 증가시키기 때문에, 모델이 특정 기울기나 시각적 패턴에만 과적합되지 않도록 도와준다.
# zoom_range=0.2: 이미지를 랜덤으로 확대 또는 축소
#                 모델이 다양한 크기와 확대 수준의 객체에 대해 더 잘 일반화할 수 있도록 돕는다.
#                 훈련 데이터에 다양한 크기의 객체를 포함시키므로, 모델이 객체 크기 변화에 대해 보다 견고하게 된다.
# -> 모델 일반화 성능을 향상시키기 위해 사용 / 과적합 방지에 유용
# -> 다양한 상황에 대해 학습할 수 있도록 돕고, 결과적으로 모델의 성능을 향상

# horizontal_flip=True: 이미지를 수평으로 뒤집는다. 데이터 증강을 통해 과적합을 방지하는 데 도움이 된다.

In [8]:
training_set = train_datagen.flow_from_directory(
    '../../data/cnn/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# target_size=(64, 64): 모든 이미지를 64x64 픽셀 크기로 변환. 이는 모델의 입력 크기와 맞추기 위함
# batch_size=32: 한 번에 처리할 이미지의 수. 일반적으로 32가 기본값으로 사용
#class_mode='binary': 이진 분류 문제이므로 'binary'로 설정. 이는 각 이미지가 두 개의 클래스 중 하나로 분류됨을 의미

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    '../../data/cnn/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
#building the CNN
cnn = keras.models.Sequential()

#Convolution
cnn.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
#Pooling
cnn.add(keras.layers.MaxPool2D(pool_size=2, strides=2))


cnn.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(keras.layers.MaxPool2D(pool_size=2, strides=2))


cnn.add(keras.layers.Flatten())


cnn.add(keras.layers.Dense(units=128, activation='relu'))


#Output Layer
cnn.add(keras.layers.Dense(units=1, activation='sigmoid')) 
# units=1 -> 최종 출력 계층의 뉴런 숫자, units 파라미터는 각 Dense 층에서 사용할 뉴런(neuron)의 수를 지정 / 이진 분류(0, 1)

model.summary()

/opt/homebrew/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# cnn.fit(x = training_set, validation_data=test_set, epochs = 25) #validation_data=cnn을 평가하려는 세트


cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.keras', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = cnn.fit(x = training_set, validation_data=test_set, epochs = 25, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - accuracy: 0.8091 - loss: 0.4082 - val_accuracy: 0.7925 - val_loss: 0.4639
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.8131 - loss: 0.3977 - val_accuracy: 0.7910 - val_loss: 0.4684
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.8248 - loss: 0.3785 - val_accuracy: 0.7930 - val_loss: 0.4750


In [16]:
from keras.preprocessing import image

# predict 메소드는 훈련에 사용했던 것과 똑같은 형식에서 불러와야 한다.
# 이미지를 불러오고 크기 조정
test_image = image.load_img('ddeock/2024_1.png', target_size=(64,64))

# pil => 이미지를 배열에 놓는 형식 / pil 이미지 인스턴스를 넘파이 배열로 변환하는 것
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
# 가짜 차원을 추가하거나 batch_size에 상응하는 차원을 추가 / 어디에 추가 차원을 넣을지 -> 항상 첫 번째 차원, 각 배치 안에 다른 이미지들이 들어간다
# 배치 차원을 추가하여 배열을 4차원으로 만듭니다. (1, 64, 64, 3) 형식

result = cnn.predict(test_image)
training_set.class_indices #class_indices: 알맞은 클래스 인덱스를 갖게 된다
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


In [17]:
prediction

'cat'